In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
import seaborn as sns
from scipy.stats import ttest_ind
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

## cp_agg

In [37]:
df = pd.read_csv('cp_agg.csv')

In [38]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,dow,year,month,day,construction,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,Number of Accident
0,2016-01-01,0:01,MANHATTAN,40.760223,-73.967462,EAST57STREET,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
1,2016-01-01,0:01,MANHATTAN,40.805057,-73.939034,EAST125STREET,4,2016,1,1,0,1.0,0.0,1,0,0,0,0,0,1
2,2016-01-01,0:05,MANHATTAN,40.799258,-73.939050,3AVENUE,4,2016,1,1,0,2.0,0.0,0,0,0,0,2,0,1
3,2016-01-01,0:15,MANHATTAN,40.738552,-73.999682,7AVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
4,2016-01-01,0:23,BROOKLYN,40.653419,-73.888970,GEORGIAAVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1


In [ ]:
cor_list =  ['dow',
 'year',
 'month',
 'day',
 'construction',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'Number of Accident']
df_cor = df[cor_list]

In [ ]:
scaler = MinMaxScaler().fit(df_cor)
trans = scaler.transform(df_cor)
trans_cor = pd.DataFrame(trans,index=df_cor.index,columns=df_cor.columns)

In [ ]:
def cor_heatmap(df):
    plt.figure(figsize=(20,10))
    cor = df.corr() #Calculate the correlation of the above variables
    sns.heatmap(cor, annot = True) #Plot the correlation as heat map
    plt.show()

In [ ]:
plt.figure(figsize=(20,10))
cor = trans_cor.corr() #Calculate the correlation of the above variables
sns.heatmap(cor, annot = True) #Plot the correlation as heat map
plt.show()

In [ ]:
# There is no linear correlation base on the heatmap above.

## Kmeans

In [ ]:
x_col = ['dow','year','month','day','construction',
         'NUMBER OF PERSONS INJURED','Number of Accident']
# x_col = ['NUMBER OF PERSONS INJURED','Number of Accident']

In [ ]:
X = trans_cor[x_col]

In [ ]:
cluster_range = range(2,5)
for n_c in cluster_range:
    silhouette = []
    km = KMeans(random_state=2000, n_clusters=n_c, n_jobs=-1)
    cluster_labels = km.fit(X).labels_
    silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
km = KMeans(random_state=2000, n_clusters=2, n_jobs=-1)
cluster_labels = km.fit(X).labels_

In [ ]:
km.cluster_centers_

In [ ]:
df['labels'] = cluster_labels

In [ ]:
label_0 = df[df['labels']==0]

In [ ]:
label_1 = df[df['labels']==1]

In [ ]:
kmean_1_0 = label_0[['CRASH DATE','NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','construction','Number of Accident']] \
    .groupby('CRASH DATE').sum().reset_index()
kmean_1_0

In [ ]:
kmean_1_1 = label_1[['CRASH DATE','NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','construction','Number of Accident']] \
    .groupby('CRASH DATE').sum().reset_index()
kmean_1_1

In [ ]:
import matplotlib.dates as mdates
fig,axs = plt.subplots(2,2,figsize=(20, 10))
# plt.locator_params(axis='x',nbins=10)
axs[0,0].plot(kmean_1_0['CRASH DATE'],kmean_1_0['construction'])
axs[0,0].axes.xaxis.set_visible(False)
axs[0,0].set_title('cluster 0: construction')

axs[0,1].plot(kmean_1_0['CRASH DATE'],kmean_1_0['Number of Accident'],c='red')
axs[0,1].axes.xaxis.set_visible(False)
axs[0,1].set_title('cluster 0: Number of Accident')

axs[1,0].plot(kmean_1_1['CRASH DATE'],kmean_1_1['construction'])
axs[1,0].axes.xaxis.set_visible(False)
axs[1,0].set_title('cluster 1: construction')

axs[1,1].plot(kmean_1_1['CRASH DATE'],kmean_1_1['Number of Accident'],c='red')
axs[1,1].axes.xaxis.set_visible(False)
axs[1,1].set_title('cluster 1: Number of Accident')
plt.show()

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [ ]:
kmean_1_0['CRASH DATE'] = pd.to_datetime(kmean_1_0['CRASH DATE'])
kmean_1_1['CRASH DATE'] = pd.to_datetime(kmean_1_1['CRASH DATE'])

In [ ]:
kmean_1_0[kmean_1_0['holiday']==True]

In [ ]:
plt.figure(figsize=(20,10))
cor = kmean_1_0.corr() #Calculate the correlation of the above variables
sns.heatmap(cor, annot = True) #Plot the correlation as heat map
plt.show()

In [ ]:
kmean_1_1[kmean_1_1['holiday']==True]

In [ ]:
plt.figure(figsize=(20,10))
cor = kmean_1_1.corr() #Calculate the correlation of the above variables
sns.heatmap(cor, annot = True) #Plot the correlation as heat map
plt.show()

In [ ]:
stat_con_acc(kmean_1_0)

In [ ]:
stat_con_acc(kmean_1_1)

In [ ]:
ttest_ind(kmean_1_0.construction,kmean_1_1.construction)

In [ ]:
ttest_ind(kmean_1_0['Number of Accident'], kmean_1_1['Number of Accident'])

In [ ]:
# Holiday analysis tells us that both clusters have same number of holidays

### Kmeans with data that aggregated by date

In [4]:
# Construction will be more related if we aggregate the data by date
df_date = df[['CRASH DATE','NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','construction','Number of Accident']] \
    .groupby('CRASH DATE').sum().reset_index()

In [ ]:
X = df_date[['construction','Number of Accident']]

In [ ]:
cluster_range = range(2,20)
for n_c in cluster_range:
    silhouette = []
    for i in range(100):
        km = KMeans(random_state=i*i, n_clusters=n_c, n_jobs=-1)
        cluster_labels = km.fit(X).labels_
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

In [ ]:
km = KMeans(random_state=2000, n_clusters=2, n_jobs=-1)
cluster_labels = km.fit(X).labels_

In [ ]:
df_date['label'] = cluster_labels
date_0 = df_date[df_date['label'] == 0]
cor_heatmap(date_0)

In [ ]:
df_date['label'] = cluster_labels
date_1 = df_date[df_date['label'] == 1]
cor_heatmap(date_1)

In [ ]:
import matplotlib.dates as mdates
fig,axs = plt.subplots(2,2,figsize=(20, 10))
# plt.locator_params(axis='x',nbins=10)
axs[0,0].plot(date_0['CRASH DATE'],date_0['construction'])
axs[0,0].axes.xaxis.set_visible(False)
axs[0,0].set_title('cluster 0: construction')

axs[0,1].plot(date_0['CRASH DATE'],date_0['Number of Accident'],c='red')
axs[0,1].axes.xaxis.set_visible(False)
axs[0,1].set_title('cluster 0: Number of Accident')

axs[1,0].plot(date_1['CRASH DATE'],date_1['construction'])
axs[1,0].axes.xaxis.set_visible(False)
axs[1,0].set_title('cluster 1: construction')

axs[1,1].plot(date_1['CRASH DATE'],date_1['Number of Accident'],c='red')
axs[1,1].axes.xaxis.set_visible(False)
axs[1,1].set_title('cluster 1: Number of Accident')
plt.show()

In [ ]:
def stat_con_acc (df):
    avg_con = df.construction.mean()
    avg_acc = df['Number of Accident'].mean()
    median_con = df.construction.median()
    median_acc = df['Number of Accident'].median()
    print('Average number of Constructions: {}'.format(avg_con))
    print('Average number of Accidents: {}'.format(avg_acc))
    print('Median number of Constructions: {}'.format(median_con))
    print('Median number of Accidents: {}'.format(median_acc))

In [ ]:
stat_con_acc(date_0)

In [ ]:
stat_con_acc(date_1)

In [ ]:
ttest_ind(date_0['construction'], date_1['construction'])

In [ ]:
ttest_ind(date_0['Number of Accident'], date_1['Number of Accident'])

### Clustering with accident and construction independently

In [5]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,dow,year,month,day,construction,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,Number of Accident
0,2016-01-01,0:01,MANHATTAN,40.760223,-73.967462,EAST57STREET,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
1,2016-01-01,0:01,MANHATTAN,40.805057,-73.939034,EAST125STREET,4,2016,1,1,0,1.0,0.0,1,0,0,0,0,0,1
2,2016-01-01,0:05,MANHATTAN,40.799258,-73.939050,3AVENUE,4,2016,1,1,0,2.0,0.0,0,0,0,0,2,0,1
3,2016-01-01,0:15,MANHATTAN,40.738552,-73.999682,7AVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
4,2016-01-01,0:23,BROOKLYN,40.653419,-73.888970,GEORGIAAVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1


In [39]:
df_acc = df[['CRASH DATE','ON STREET NAME','Number of Accident','construction']]
df_acc['ON STREET NAME'] = df_acc['ON STREET NAME'].astype('category')
df_acc['ON STREET NAME'] = df_acc['ON STREET NAME'].cat.codes
df_acc

,CRASH DATE,ON STREET NAME,Number of Accident,construction
0,2016-01-01,2246,1,0
1,2016-01-01,2073,1,0
2,2016-01-01,472,1,0
3,2016-01-01,658,1,0
4,2016-01-01,2647,1,0
...,...,...,...,...
571569,2019-12-31,565,1,0
571570,2019-12-31,5022,1,0
571571,2019-12-31,4157,1,0
571572,2019-12-31,886,1,0


In [41]:
# Standardize
# scaler = MinMaxScaler().fit(df_acc)
# trans = scaler.transform(df_acc)
# acc_trans = pd.DataFrame(trans,index=df_acc.index,columns=df_acc.columns)
# acc_trans

In [10]:
# X = acc_trans[['ON STREET NAME','Number of Accident']]
X = df_acc[['Number of Accident']]
cluster_range = range(2,20)
for n_c in cluster_range:
    silhouette = []
    km = KMeans(random_state=2000, n_clusters=n_c, n_jobs=-1)
    cluster_labels = km.fit(X).labels_
    silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

In [11]:
km = KMeans(random_state=2000, n_clusters=3, n_jobs=-1)
cluster_labels = km.fit(X).labels_

In [13]:
df_acc['labels'] = cluster_labels

In [34]:
df_0 = df_acc[df_acc['labels']==0]


570108

In [35]:
df_1 = df_acc[df_acc['labels']==1]
len(df_1)

1424

In [36]:
df_2 = df_acc[df_acc['labels']==2]
len(df_2)

42

### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import operator

In [ ]:
df_date = df[['CRASH DATE','construction','Number of Accident']] \
    .groupby('CRASH DATE').sum().reset_index()
df_date['CRASH DATE'] = pd.to_datetime(df_date['CRASH DATE'])

In [ ]:
df_date['dow'] = df_date['CRASH DATE'].dt.dayofweek
df_date['year'] = df_date['CRASH DATE'].dt.year
df_date['month'] = df_date['CRASH DATE'].dt.month
df_date['day'] = df_date['CRASH DATE'].dt.day
model_train = df_date[df_date['year']!=2019]
model_train.head(5)

In [ ]:
model_test = df_date[df_date['year']==2019]
model_test.head(5)

In [ ]:
X = model_train[['construction','dow','year','month','day']]
y = model_train['Number of Accident']

X_train = model_train[['construction','dow','year','month','day']]
y_train = model_train['Number of Accident']

X_test = model_test[['construction','dow','year','month','day']]
y_test = model_test['Number of Accident']

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020,shuffle=False)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,random_state=2020,shuffle=False)

rf = RandomForestRegressor(n_estimators=1000, random_state=0)
rf.fit(X_train, y_train)

In [ ]:
# 
rf.score(X_test, y_test)

In [ ]:
depth_R2 = {}
for depth in np.linspace(2, 2000, 100): # try 2-10 depths
    
    rf = RandomForestRegressor(max_depth=depth)
    rf.fit(X_train, y_train)

    depth_R2[depth] = rf.score(X_val, y_val)
    
print(f'Best max_depth: ', max(depth_R2.items(), key=operator.itemgetter(1))[0])

In [ ]:
# optimize the max_depth parameter
leaf_R2 = {}
for nodes in np.linspace(20, 2000, 100): 
    
    rf = RandomForestRegressor(max_depth=206, max_leaf_nodes=int(nodes))
    rf.fit(X_train, y_train)

    leaf_R2[nodes] = rf.score(X_val, y_val)
print(f'Best leaf_nodes: ', max(leaf_R2.items(), key=operator.itemgetter(1))[0])

In [ ]:
param_grid = {'kpca__gamma':np.linspace(0.001, 0.1, 20)}
cv = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=5,cv=4,scoring='accuracy') 

## Limitation of this Dataset

The performance of the Kmeans model shows the limitation of our dataset. The dataset was acquired using the NYC construction petmit and Motor Vehicle collisions dataset. Our initial assumptions are the dateset is missing informations about the length of the street or the amount of activity of a street, and we got the number of constructions using the IssuedWorkStartDate and IssuedWorkEndDate. These two features do not tell if there were constructions actually happening on the street. Thus, the dateset will not give me perfect analysis on the relationship between constructions and accidents, and the result matches our assumptions.

## Holiday Analysis

In [ ]:
dr = pd.date_range(start='2016-01-01', end='2019-12-31')
cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
df_holiday = df.copy()

In [ ]:
df_holiday['CRASH DATE'] = pd.to_datetime(df_holiday['CRASH DATE'])
df_holiday['holiday'] = df_holiday['CRASH DATE'].isin(holidays)

In [ ]:
df_holiday['holiday'] = df_holiday['holiday'].replace(True,1)
df_holiday['holiday'] = df_holiday['holiday'].replace(False,0)
# holiday_0 = df_holiday[df_holiday['holiday']==0]
# holiday_1 = df_holiday[df_holiday['holiday']==1]

In [ ]:
# fig,axs = plt.subplots(2,2,figsize=(20, 10))
# # plt.locator_params(axis='x',nbins=10)
# axs[0,0].plot(holiday_0['CRASH DATE'],holiday_0['construction'])
# axs[0,0].axes.xaxis.set_visible(False)
# axs[0,0].set_title('cluster 0: construction')

# axs[0,1].plot(holiday_0['CRASH DATE'],holiday_0['Number of Accident'],c='red')
# axs[0,1].axes.xaxis.set_visible(False)
# axs[0,1].set_title('cluster 0: Number of Accident')

# axs[1,0].plot(holiday_1['CRASH DATE'],holiday_1['construction'])
# axs[1,0].axes.xaxis.set_visible(False)
# axs[1,0].set_title('cluster 1: construction')

# axs[1,1].plot(holiday_1['CRASH DATE'],holiday_1['Number of Accident'],c='red')
# axs[1,1].axes.xaxis.set_visible(False)
# axs[1,1].set_title('cluster 1: Number of Accident')
# plt.show()

In [ ]:
X = df_holiday[['construction','Number of Accident','holiday']]
cluster_range = range(2,20)
for n_c in cluster_range:
    silhouette = []
    for i in range(100):
        km = KMeans(random_state=i*i, n_clusters=n_c, n_jobs=-1)
        cluster_labels = km.fit(X).labels_
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))

### Old dataset

In [ ]:
df_crash_permit = pd.read_csv('crash_permit.csv')
df_crash_permit